# 03 Function Calling | Tools

## Step 1: Create Search Index Client




In [1]:
#r "nuget: Azure.AI.OpenAI, 2.0.0-beta.2"
#r "nuget: DotNetEnv, 2.5.0"

using Azure; 
using Azure.AI.OpenAI;
using OpenAI.Chat;
using DotNetEnv;
using System.IO;
using System.Text.Json; 
using System.ClientModel;

//configuration file is created during environment creation
static string configurationFile = @"../Configuration/application.env";
Env.Load(configurationFile);

string oAiApiKey = Environment.GetEnvironmentVariable("WS_AOAI_APIKEY") ?? "WS_AOAI_APIKEY not found";
string oAiEndpoint = Environment.GetEnvironmentVariable("WS_AOAI_ENDPOINT") ?? "WS_AOAI_ENDPOINT not found";
string chatCompletionDeploymentName = Environment.GetEnvironmentVariable("WS_CHATCOMPLETION_DEPLOYMENTNAME") ?? "WS_CHATCOMPLETION_DEPLOYMENTNAME not found";


ApiKeyCredential apiKeyCredential = new ApiKeyCredential(oAiApiKey);
AzureOpenAIClient azureOpenAIClient = new AzureOpenAIClient(new Uri(oAiEndpoint), apiKeyCredential);
ChatClient chatClient = azureOpenAIClient.GetChatClient(chatCompletionDeploymentName);

Console.WriteLine($"AzureOpenAI Client created...");
Console.WriteLine($"ChatClient created...");

Installed Packages Azure.AI.OpenAI, 2.0.0-beta.2 DotNetEnv, 2.5.0

AzureOpenAI Client created...
ChatClient created...


# Step 2: Define Tools




In [2]:
private string GetSportEventWinner(string sportEvent, string year)
{
    // Implement the logic to get the winner of the sport event.
    return "Kansas City Chiefs";
}

private string GetSportEventResult(string sportEvent, string year)
{
    // Implement the logic to get the result of the sport event.
    return "24:1";
}

ChatTool getSportEventWinnerTool = ChatTool.CreateFunctionTool(
    functionName: nameof(GetSportEventWinner),
    functionDescription: "Get the winner of a sport Event",
    functionParameters: BinaryData.FromString("""
    {
        "type": "object",
        "properties": {
            "sportEvent": {
                "type": "string",
                "description": "The name of the sport event"
            },
            "year": {
                "type": "string",
                "description": "The year in which the sport event took place"
            }
        },
        "required": [ "sportEvent", "year" ]
    }
    """)
);

ChatTool getSportEventResultTool = ChatTool.CreateFunctionTool(
    functionName: nameof(GetSportEventResult),
    functionDescription: "Get the result of a sport Event",
    functionParameters: BinaryData.FromString("""
    {
        "type": "object",
        "properties": {
            "sportEvent": {
                "type": "string",
                "description": "The name of the sport event"
            },
            "year": {
                "type": "string",
                "description": "The year in which the sport event took place"
            }
        },
        "required": [ "sportEvent", "year" ]
    }
    """)
);

private string CallLocalFunctions(ChatToolCall toolCall)
{
    //Consider adding error handling & function calling using reflection
    using JsonDocument argumentsDocument = JsonDocument.Parse(toolCall.FunctionArguments);
    switch (toolCall.FunctionName)
    {
        case nameof(GetSportEventWinner): {
            string sportEvent = argumentsDocument.RootElement.GetProperty("sportEvent").GetString() ?? "";
            string year = argumentsDocument.RootElement.GetProperty("year").GetString() ?? "";

            Console.WriteLine($"GetSportEventWinner({sportEvent}, {year}) - Called");
            return GetSportEventWinner(sportEvent, year);
        }
        case nameof(GetSportEventResult): {
            string sportEvent = argumentsDocument.RootElement.GetProperty("sportEvent").GetString() ?? "";
            string year = argumentsDocument.RootElement.GetProperty("year").GetString() ?? "";

            Console.WriteLine($"GetSportEventResult({sportEvent}, {year}) - Called");
            return GetSportEventResult(sportEvent, year);
        }

        default:
            return "Function not defined!"; 
    }
}

Console.WriteLine("Functions and corresponding tools definition created...");

Functions and corresponding tools definition created...


## Step 3: Define Chat Completion Options & Chat Messages

In [9]:
//Define Chat Completion Options
ChatCompletionOptions chatCompletionOptions = new()
{
    Tools = { getSportEventResultTool, getSportEventWinnerTool },
};

//Define Chat Messages
string systemMessage = ""; 
List<string> userMessages = new List<string> { "Who won the Super Bowl in 2024? And what was the result?" };
List<ChatMessage> chatMessages = new List<ChatMessage>();

chatMessages.Add(new SystemChatMessage(systemMessage)); 
chatMessages.AddRange(userMessages.Select(message => new UserChatMessage(message)));

Console.WriteLine("Chat Messages created...");

Chat Messages created...


## Step 4: Call Chat Completion

In [13]:
//Call chat completion (with tools)
ChatCompletion chatCompletion = await chatClient.CompleteChatAsync(
    messages: chatMessages,
    options: chatCompletionOptions
);

//Check Response
while (chatCompletion.FinishReason == ChatFinishReason.ToolCalls)
{
    // Add a new assistant message to the conversation history that includes the tool calls
    chatMessages.Add(new AssistantChatMessage(chatCompletion));

    foreach (ChatToolCall toolCall in chatCompletion.ToolCalls)
    {
        chatMessages.Add(
            new ToolChatMessage(
                toolCall.Id, 
                CallLocalFunctions(toolCall)
            )
        );
    }

    chatCompletion = await chatClient.CompleteChatAsync(
        messages: chatMessages,
        options: chatCompletionOptions
    );
}

foreach (ChatMessageContentPart chatMessageContentPart in chatCompletion.Content){
    Console.WriteLine($"Model response: \n {chatMessageContentPart}"); 
}

Model response: 
 The Kansas City Chiefs won the Super Bowl in 2024. The result of the game was 24-1.
